In [38]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
# 載入使用的套件
import pandas as pd
import requests
import json
import re
urlarray = []
infos = []
# 撰寫簡單的函數，透過輸入文章ID，就輸出文章的資料
def Crawl():
  info = {}
  link = 'https://www.dcard.tw/_api/posts/' + str(ID)
  requ = requests.get(link)
  rejs = json.loads(requ.text)
  for d in rejs:
    info['ID'] = d['id']
    info['title'] = d['title']
    info['excerpt'] = d['excerpt']
    info['forumName'] = d['forumName']
  infos.append(info)



url = 'https://www.dcard.tw/_api/posts?limit=100'
resq = requests.get(url)
rejs = json.loads(resq.text)
for d in rejs:
  info = {}
  info['ID'] = d['id']
  info['title'] = d['title']
  info['excerpt'] = d['excerpt']
  info['forumName'] = d['forumName']
  infos.append(info)

df = pd.DataFrame(infos)
df.columns = ['ID','title','excerpt','forumName']
print(df)
# df.to_csv('dcard.csv', encoding='utf_8_sig')  

           ID                                          title  \
0   236793339                               #請認領 每日人像練習🍓8/23   
1   236793336                                   家將街舞環島公益教學計畫   
2   236793333                               #iPad iPad螢幕出現黑條   
3   236793332                                       我們都認識自己嗎   
4   236793330                                 想知道165女生的腿圍 腰圍   
5   236793329                            #教檢 準備兩個月分享（選擇題分數低）   
6   236793327                                   約出門最討厭的那種朋友😡   
7   236793325                              收/售 素材 非賣品 藝術品 照片   
8   236793324                                         淘寶相關問題   
9   236793322                                       請問你們認同嗎？   
10  236793321                                          台綜A10   
11  236793320                                       很在意男友說的話   
12  236793317                                    #招租 求～北北基租屋   
13  236793316                      #分潤 雷蛇 幻彩電競鍵盤 券後只要$4640‼️   
14  236793315                           

In [58]:
# 載入使用的套件
import pandas as pd
import requests
import json
import re
import requests
from bs4 import BeautifulSoup

infos = []
# 撰寫簡單的函數，透過輸入文章ID，就輸出文章的資料
def getdata(link):
  info = {}
  requ = requests.get(link)
  print(requ)
  rejs = json.loads(requ.text)
  for d in rejs:
    info['ID'] = d['id']
    info['title'] = d['title']
    info['excerpt'] = d['excerpt']
    info['forumName'] = d['forumName']
  infos.append(info)

headers = {
      'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'
  }

#231997531
for i in range(1,101):
  url = 'https://www.dcard.tw/service/api/v2/posts//%d'%(i+231997431)+''
  # getdata(url)
  resq = requests.get(url,headers=headers)
  rejs = json.loads(resq.text)
  for d in rejs:
    info = {}
    info['ID'] = d['id']
    info['title'] = d['title']
    info['excerpt'] = d['excerpt']
    info['forumName'] = d['forumName']
    infos.append(info)

df = pd.DataFrame(infos)
df.columns = ['ID','title','excerpt','forumName']
print(df)
# df.to_csv('dcard.csv', encoding='utf_8_sig')  

JSONDecodeError: ignored

In [ ]:
#hw3

import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split    #分割資料集

def cutval(df):
    topcut = []
    for d in df['劇情介紹']:
        top = jieba.analyse.extract_tags(d,topK=2)
        topcut.append(top)
    typecut = []
    for d in df['oneLabel']:
        # ch =re.compile("[\u4e00-\u9fa5]")
        # seg_word =  "".join(ch.findall(d))
        # top = jieba.lcut(seg_word)
        typecut.append(d)
    # print(typecut)
    namecut = []
    for d in df['中文名稱']:
        ch =re.compile("[\u4e00-\u9fa5]")
        name =  "".join(ch.findall(d))
        # print("name = ")
        # print(name )
        namecut.append(name)
    # print(namecut)
    data = { 'type':typecut ,'name':namecut,'article':topcut}
    df1 = pd.DataFrame(data)

    df1['type'] = df1['type'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['article'] = df1['article'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['key'] = df1['name'].astype(str)+' '+df1['type'].astype(str)+' '+df1['article'].astype(str)
    return df1

def count(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['key'])
    tfidf = TfidfTransformer() 
    tf=tfidf.fit_transform(X)
    word = vectorizer.get_feature_names() #詞袋 
    print("tf = ")
    print(tf)
    return tf

def knn(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['type'].str[0:2], test_size=0.07)
    clf=KNeighborsClassifier(n_neighbors=51)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)    # 預測模型 
    y_test=y_test.values
    print("精準度：",metrics.accuracy_score(y_test, y_pred))

df = pd.read_csv('/content/drive/MyDrive/NatureLanguageProgram/B0729064_NLP_HW3/movies.csv')
df = df.drop(labels=['Unnamed: 0'],axis='columns')
df1 = cutval(df) 
tf = count(df1)
knn(tf,df1)